# Clinical Workflow Integration

In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize
from keras.models import model_from_json
import glob

Using TensorFlow backend.


## Check DICOM header for proper image acquisition type, position, and body part  

In [2]:
## First, read all of my DICOM files into a list
mydicoms = glob.glob("*.dcm")

In [3]:
## Let's look at the contents of the first DICOM:
dcm1 = pydicom.dcmread(mydicoms[0])
dcm1

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Secondary Capture Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.8.498.1
(0002, 0013) Implementation Version Name         SH: 'PYDICOM 1.2.0'
-------------------------------------------------
(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patien

In [4]:
## Create the dataframe that we want, and populate it in a loop with all of our DICOMS
all_data = []

for i in mydicoms: 
    dcm = pydicom.dcmread(i)
    fields = [dcm.Modality, dcm.PatientPosition, dcm.BodyPartExamined]
    all_data.append(fields)
    
mydata = pd.DataFrame(all_data, 
                      columns = ['Modality','Patient Position','Body Part Examined'])
mydata

,Modality,Patient Position,Body Part Examined
0,DX,PA,CHEST
1,DX,AP,CHEST
2,DX,AP,CHEST
3,DX,PA,RIBCAGE
4,CT,PA,CHEST
5,DX,XX,CHEST


Notice the CT scan (Image 4), the XX patient position (Image 5), and the Ribcage (Image 3). We will check these parameters on the check_dicom function. 

## Read imaging data from a .dcm file, preprocess the image and feed it into the model

In [5]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
          
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    if (ds.BodyPartExamined !='CHEST') | (ds.Modality !='DX') | (ds.PatientPosition not in ['PA','AP']):
          print('The model is not valid - wrong image position, image type, or body part.')
    
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
    proc_img = (img - img_mean)/img_std
    proc_img =  resize(proc_img, img_size, anti_aliasing=True) # Resize image to match a certain size.
        
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
          
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
     
    result = model.predict(img)  
    predict=result[0]
    prediction='No pneumonia'
    if(predict>thresh):
        prediction='Pneumonia'
   
    return prediction 

In [ ]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" # path to saved model
weight_path = "xray_class_model.best.hdf5" # path to saved best weights

IMG_SIZE = (1,224,224,3) # This might be different if you did not use vgg16
#img_mean = # loads the mean image value they used during training preprocessing
#img_std = # loads the std dev image value they used during training preprocessing

# Here I calculate the mean and std of the given images: 
for file in test_dicoms:
    ds = pydicom.dcmread(file)       
    img = ds.pixel_array
    img_mean = np.mean(img)
    img_std = np.std(img)
img_mean = np.mean(img_mean)
img_std = np.mean(img_std)


json_file = open(model_path, "r") 
loaded_model_json = json_file.read() 
json_file.close() 
my_model = model_from_json(loaded_model_json) #loads model
thresh = 0.54965 #threshold chosen for model classification (maxes F1-score maximum)

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)